[comment]: <> (Written by Andrés Cárdenas)
[comment]: <> (July 2021)
[comment]: <> (www.fysixs.com)
[comment]: <> (andres.cardenas@gmail.com)

<div>

 <img src="https://drive.google.com/thumbnail?id=1cJN2J8ByFPCfATK70k6CiHeP0bKGN-Ii" alt="Snow" width="35"> <font face="Gill Sans" color = #667495> &nbsp;**A Julia/Jupyter Companion**
 <img src = "https://i.imgur.com/Bg81qo9.png?2" width = 70 align="right"><img src="https://drive.google.com/thumbnail?id=1nmgz_xGqeqgvU8wBfJFERu1Zs82bBZ84" alt="Snow" width="55" align ="right"> 
 <br>
  *Andrés Cárdenas, 2021*, <i><a href="https://www.fysixs.com">www.fysixs.com</a></i>
</font>
</div>
<hr size=5 color=#8D84B5 > </hr> 

<div align="right"><br>
    
<font color = #667495 face="Gill Sans" size = 4>***Principles of Quantum Mechanics*** <br><font color = #667495 face="Gill Sans" size=3> by R. Shankar <font size=2> (<a href="https://www.amazon.com/Principles-Quantum-Mechanics-R-Shankar-ebook/dp/B000SEIXA2">text on Amazon</a>)</font> </font><br> </div>
    
<font color="#8D84B5" face="Gill Sans" size = 5> <b>SOURCE FILE FOR THE BOOK</b></font>
<hr size=5 color=#8D84B5 > </hr> 

---
### <font color="#8D84B5"> Imports

In [ ]:
using LinearAlgebra
using StaticArrays
using SymPy
using Plots
using LaTeXStrings

gr()
Plots.GRBackend()

---
### <font color="#8D84B5"> Julia Environment Setup

In [ ]:
# Symbolic declarations
@syms θ::real
@syms α₁, α₂, α₃
@syms θ₁::real, θ₂::real, θ₃::real
@syms ω, Ω₁₁, Ω₁₂, Ω₂₁, Ω₂₂
@syms x₁, x₂, x₃

# Bra-ket types
const ket{N, T} = SArray{Tuple{N, 1}, T, 2, N};
const bra{N, T} = SArray{Tuple{1, N}, T, 2, N};
@inline ket(x::NTuple{N,Any})  where {N}    = ket{N}(x)
@inline ket{N}(x::NTuple{N,T}) where {N, T} = ket{N, T}(x)
@inline bra(x::NTuple{N,Any})  where {N}    = bra{N}(x)
@inline bra{N}(x::NTuple{N,T}) where {N, T} = bra{N, T}(x)

# Define these basis elements here
î = ket(1, 0, 0)
ĵ = ket(0, 1, 0)
k̂ = ket(0, 0, 1)

# Add an array dispatch to simplify and expand
import SymPy: simplify, expand
function simplify(A::T) where {T <: AbstractArray}
    szA = size( A )
    
    if szA |> length == 2
        n = szA[1]
        m = szA[2]
        vals = [ simplify(A[i,j]) for j ∈ 1:m for i ∈ 1:n ]
    elseif szA |> length == 1
        n = szA[1]
        vals = [ simplify(A[i]) for i ∈ 1:n ]
    end
    
    if typeof(A) <: StaticArray
        Aout = typeof(A)( Tuple( Sym(x) for x in vals ) )
    else
        if szA |> length == 2
            Aout   = typeof(A)( undef, n, m )
            k = 1
            for j ∈ 1:m
                for i ∈ 1:n
                    Aout[i,j] = Sym(vals[k])
                    k += 1
                end
            end
        else
            Aout = typeof(A)( [ Sym(x) for x in vals ] )
        end
    end
        
    return Aout
end

function expand(A::T) where {T <: AbstractArray}
    szA = size( A )
    
    if szA |> length == 2
        n = szA[1]
        m = szA[2]
        vals = [ expand(A[i,j]) for j ∈ 1:m for i ∈ 1:n ]
    elseif szA |> length == 1
        n = szA[1]
        vals = [ expand(A[i]) for i ∈ 1:n ]
    end
    
    if typeof(A) <: StaticArray
        Aout = typeof(A)( Tuple( Sym(x) for x in vals ) )
    else
        if szA |> length == 2
            Aout   = typeof(A)( undef, n, m )
            k = 1
            for j ∈ 1:m
                for i ∈ 1:n
                    Aout[i,j] = Sym(vals[k])
                    k += 1
                end
            end
        else
            Aout = typeof(A)( [ Sym(x) for x in vals ] )
        end
    end
        
    return Aout
end;

# Better TeX printing for the REPL
TeX(expr) = expr |> sympy.latex |> latexstring;
disp(str) = display("text/markdown", str);

---

### <font color="#8D84B5"> Mathematical Definitions

In [ ]:
import Base: |
"""
Define the inner product by 
overloading the vertical bar to make bra-kets
"""
|(a::ket, b::ket) = (a'*b)[1];
|(a::bra, b::ket) = (a*b)[1];

"""
Define the commutator between operators Ω and Λ
"""
Com(Ω, Λ) = (Ω)Λ - (Λ)Ω

"""
Create a unit ket
"""
function uvec(a::ket)
    â = a / sqrt( a|a )
    return â .|> Sym
end

"""
Create a unit bra
"""
function uvec(b::bra)
    b̂ = b / sqrt( b|b' )
    return b̂ .|> Sym
end

"""
Gram-Schmidt Algorithm
"""
function gramschmidt(ℬ)
    ℬ′ = typeof(ℬ)()
    
    push!( ℬ′, uvec( ℬ[1] ) )
    for i ∈ 2:length(ℬ)
        ℬ′ᵢ = ℬ[i]
        for j ∈ 1:i-1
            ℬ′ᵢ += - (ℬ′[j]|ℬ[i])ℬ′[j]
        end
        push!( ℬ′, uvec( ℬ′ᵢ ) )
    end
    return ℬ′
end

"""
Rotation Operators
3D rotation matrix about axis n̂
by an angle θ
"""
function R(θ, n̂::ket)
    R₀ = @SMatrix[
        1    0               0;
        0    cos(θ)          -sin(θ);
        0    sin(θ)          cos(θ);
        ] 
    R₁ = @SMatrix[
        cos(θ)    0          -sin(θ);
        0         1          0;  
        sin(θ)    0          cos(θ);
        ] 
    R₂ = @SMatrix[
        cos(θ)    -sin(θ)    0;
        sin(θ)    cos(θ)     0;
        0         0          1;
        ]
        
    î = ket(1, 0, 0)
    ĵ = ket(0, 1, 0)
    k̂ = ket(0, 0, 1)

    if (n̂ == î)
        op = R₀
    elseif (n̂ == ĵ)
        op = R₁
    elseif (n̂ == k̂)
        op = R₂
    end 
    
    return op
end

"""
Obtain the matrix elements of an operator in a given basis
"""
function mat( Ω, ℬ )
    dim  = length(ℬ)
    vals = [ ( ℬ[i]|(Ω*ℬ[j]) ) for j ∈ 1:dim for i ∈ 1:dim ]
    mat  = typeof(Ω)( Tuple( Sym(x) for x in vals ) )
    return mat
end

"""
Projection operator for ket V
"""
ℙ(V::ket) = (V)V'

In [ ]:
#END SOURCE